<a href="https://colab.research.google.com/github/aledima00/Project4_SemSeg_AML2024/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 4 - Semantic Segmentation

First let's download dataset, that is already split in "Train", "Test" and "Val" modules

In [1]:
!pip install icecream

In [48]:
import os
import glob
from icecream import ic
from skimage.io import imread
import logging
from enum import Enum
import gdown

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms

from PIL import Image

import numpy as np

General Configuration:

In [3]:
DBG = True                    # set to True for debug mode (lighter execution + dbg prints)
CONFIG_DATASET = True         # set to True to download and config all dataset resources
CONFIG_DEEPLABV2 = True       # set to True to download and config all DeepLabv2 resources

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")
TRAIN_PATH = "Train"          # path of the train folder
VAL_PATH = "Val"              # path of the val folder
TEST_PATH = "Test"            # path of the test folder
DEEPLABV2_WEIGHTS_PATH = "deeplabv2-pretrain-weights.pth"  # path of the deeplabv2 folder

Using device: cuda


download datasets:

In [4]:
def config_generic_dataset(DS_PATHNAME,URL):
  !rm -rf {DS_PATHNAME}
  ZIP_PATH = DS_PATHNAME + ".zip"
  !rm {ZIP_PATH}
  !wget -O {ZIP_PATH} {URL}
  !unzip {ZIP_PATH} | tail -n 3
  !rm {ZIP_PATH}

def config_train_dataset():
  config_generic_dataset(TRAIN_PATH, "https://zenodo.org/records/5706578/files/Train.zip?download=1")
def config_val_dataset():
  config_generic_dataset(VAL_PATH, "https://zenodo.org/records/5706578/files/Val.zip?download=1")
def config_test_dataset():
  config_generic_dataset(TEST_PATH, "https://zenodo.org/records/5706578/files/Test.zip?download=1")

def config_all_dataset(*,force=False):
  print("Dowloading and Configuring Dataset")
  if force or (not os.path.exists(TRAIN_PATH)):
    config_train_dataset()
  if force or (not os.path.exists(VAL_PATH)):
    config_val_dataset()
  if force or (not os.path.exists(TEST_PATH)):
    config_test_dataset()

download and configure deeplabv2 model library (_with R101 backbone_) and the pretrain weights:

In [5]:
def config_deeplabv2_model():
  print("Dowloading and Configuring DeepLabv2 Model")
  import sys
  import gdown
  !rm -rf "MLDL2024_project1"
  !git clone https://github.com/Gabrysse/MLDL2024_project1.git
  sys.path.append("/content/MLDL2024_project1/")
  gdown.download("https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v", DEEPLABV2_WEIGHTS_PATH, quiet=False)


In [6]:
if CONFIG_DATASET:
  config_all_dataset()
if CONFIG_DEEPLABV2:
  config_deeplabv2_model()

Dowloading and Configuring Dataset
rm: cannot remove 'Train.zip': No such file or directory
--2024-12-18 18:54:51--  https://zenodo.org/records/5706578/files/Train.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4021669263 (3.7G) [application/octet-stream]
Saving to: ‘Train.zip’

Train.zip           100%[===================>]   3.75G  71.4MB/s    in 66s     

2024-12-18 18:55:58 (58.0 MB/s) - ‘Train.zip’ saved [4021669263/4021669263]

  inflating: Train/Urban/masks_png/2519.png  
  inflating: Train/Urban/masks_png/2520.png  
  inflating: Train/Urban/masks_png/2521.png  
rm: cannot remove 'Val.zip': No such file or directory
--2024-12-18 18:56:43--  https://zenodo.org/records/5706578/files/Val.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to ze

Downloading...
From (original): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v
From (redirected): https://drive.google.com/uc?id=1ZX0UCXvJwqd2uBGCX7LI2n-DfMg3t74v&confirm=t&uuid=51eaf9de-153d-4466-8218-6d237b4f3794
To: /content/deeplabv2-pretrain-weights.pth
100%|██████████| 177M/177M [00:01<00:00, 89.7MB/s]


## Data Preparation

Create Dataset class and filter urban pictures...

In [7]:
NUM_CLASSES = 7
BATCH_SIZE = 2 if DBG else 128
LR = 0.001           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 4e-5  # Regularization, you can keep this at the default
NUM_EPOCHS = 20      # Total number of training epochs (iterations over dataset)
STEP_SIZE = [25, 75, 150] # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

In [55]:
# Define transforms for training phase
train_image_transform = transforms.Compose([
    transforms.Resize(256),      # Resizes short size of the PIL image to 256
    transforms.CenterCrop(224),  # Crops a central square patch of the image
                                  # 224 because torchvision's AlexNet needs a 224x224 input!
                                  # Remember this when applying different transformations, otherwise you get an error
    transforms.ToTensor(), # Turn PIL Image to torch.Tensor
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # Normalize as per ImageNet stats
])

# Define transforms for the evaluation phase
test_image_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # Normalize as per ImageNet stats
])

# Masks should not be normalized or resized with interpolation
target_transform = transforms.Compose([
    transforms.Resize((256, 256), interpolation=Image.NEAREST),
    transforms.CenterCrop(224),
    transforms.Lambda(lambda mask: torch.tensor(np.array(mask), dtype=torch.uint8)),
])


In [24]:
# taken from official repo of LoveDA
COLOR_MAP = {
    0:"IGNORE",
    1:"Background",
    2:"Building",
    3:"Road",
    4:"Water",
    5:"Barren",
    6:"Forest",
    7:"Agricultural"
}

In [56]:
def pil_loader(path,*,format:str="RGB"):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert(format)

class DataType(Enum):
  RURAL = 0
  URBAN = 1

class LoveDA(Dataset):
  def __init__(self, basedir, data_type:DataType, transforms=None, target_transform=None):
    #super(LoveDA, self).__init__(basedir, transforms, target_transform) # should we do this??
    if data_type == DataType.RURAL:
        self.base_path = os.path.join(basedir, "Rural")
    else: #data_type == DataType.URBAN:
        self.base_path = os.path.join(basedir, "Urban")


    # list of integers that identifies paths relative to both images_png and masks_png
    self.int_pathrefs = os.listdir(os.path.join(self.base_path, "images_png"))
    self.int_pathrefs = list(int(st.split(".")[0]) for st in self.int_pathrefs)

    # -------------------------- DEBUG --------------------------
    if DBG:
      self.int_pathrefs = self.int_pathrefs[:15] # limit the dataset for debug
      ic(self.int_pathrefs)
    # -------------------------- DEBUG --------------------------

    self.transforms = transforms
    self.target_transform = target_transform

  def __getitem__(self, idx):
    assert idx < len(self), 'Index out of range'
    image_path = os.path.join(self.base_path, "images_png", str(self.int_pathrefs[idx]) + ".png")
    mask_path = os.path.join(self.base_path, "masks_png", str(self.int_pathrefs[idx]) + ".png")
    image = pil_loader(image_path,format="RGB")
    mask = pil_loader(mask_path,format="L")

    if self.transforms is not None:
      image = self.transforms(image)
    if self.target_transform is not None:
      mask = self.target_transform(mask) # should we apply transformation on mask ??

    #image,mask = torch.tensor(image),torch.tensor(mask) # to torch tensors
    #mask = torch.tensor(mask, dtype=torch.uint8)

    # -------------------------- DEBUG --------------------------
    if DBG:
      print("post-transform:")
      ic(image)
      ic(mask)
    # -------------------------- DEBUG --------------------------

    return image, mask

  def __len__(self):
    return len(self.int_pathrefs)


In [57]:
# Urban Datasets (train, val, test)
urban_train = LoveDA(TRAIN_PATH, DataType.URBAN, transforms=train_image_transform, target_transform=target_transform)
urban_val = LoveDA(VAL_PATH, DataType.URBAN, transforms=test_image_transform, target_transform=target_transform)
urban_test = LoveDA(TEST_PATH, DataType.URBAN, transforms=test_image_transform, target_transform=target_transform)

# Urban Dataloaders (train, val, test)

NUM_WORKERS = 1 if DBG else 4
urban_train_dataloader = DataLoader(urban_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, drop_last=True)
urban_val_dataloader = DataLoader(urban_val, shuffle=False, num_workers=NUM_WORKERS, drop_last=False)
urban_test_dataloader = DataLoader(urban_test, shuffle=False, num_workers=NUM_WORKERS, drop_last=False)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Get DeepLabv2 model with pretrain weights:

In [58]:
from MLDL2024_project1.models.deeplabv2 import deeplabv2

model = deeplabv2.get_deeplab_v2(num_classes=7,pretrain=True,pretrain_model_path=DEEPLABV2_WEIGHTS_PATH)

Deeplab pretraining loading...


/content/MLDL2024_project1/models/deeplabv2/deeplabv2.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  saved_state_dict = torch.load(pretrain_model_path)


## Optimizer, Loss, ecc.

In [59]:
# enable validation during training
validate = True

model.train(True)
model.multi_level = False # ask in class
for params in model.get_1x_lr_params_no_scale():
  params.requires_grad = False # no training in Backbone
for params in model.get_10x_lr_params():
  params.requires_grad = True # training in classifiers


model = model.to(DEVICE) # switch to GPU

#Loss (as said in DeepLabv2 docs)
loss_fn = nn.CrossEntropyLoss()

#Opt
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer_fn = optim.Adam(trainable_params, lr=LR, weight_decay=WEIGHT_DECAY, eps=1e-4)

#Scheduler
optim_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=STEP_SIZE, gamma=GAMMA)


## Training Loop

In [69]:
from colorama import Fore, Back
DBG = False
for inputs, targets in urban_train_dataloader:
  print(inputs)
  print(targets)
  break

tensor([[[[-0.3198, -0.4568, -0.9877,  ...,  0.0741,  0.4166,  0.6049],
          [-0.5253, -0.5596, -0.5253,  ..., -0.2171,  0.2282,  0.5707],
          [-0.3883, -0.2513, -0.3541,  ..., -0.5767, -0.1828,  0.3652],
          ...,
          [-0.2856, -0.3541, -0.3541,  ..., -1.0219, -0.8507, -0.5767],
          [-0.2856, -0.3712, -0.3712,  ..., -0.5938, -0.5082, -0.4911],
          [-0.2342, -0.3541, -0.3712,  ..., -0.0629, -0.4397, -0.4226]],

         [[-0.1800, -0.2325, -0.7402,  ..., -0.0399,  0.2402,  0.3627],
          [-0.3901, -0.4076, -0.3375,  ..., -0.3025,  0.0826,  0.3277],
          [-0.2150, -0.0924, -0.1800,  ..., -0.5826, -0.3025,  0.1527],
          ...,
          [-0.1099, -0.1450, -0.1450,  ..., -0.7227, -0.6001, -0.3550],
          [-0.1099, -0.1450, -0.1625,  ..., -0.3375, -0.2675, -0.2325],
          [-0.0574, -0.1275, -0.1450,  ...,  0.2402, -0.1450, -0.1275]],

         [[ 0.0256, -0.0615, -0.5670,  ...,  0.0082,  0.1825,  0.2696],
          [-0.2010, -0.2881, -

In [67]:
import warnings
from torch.backends import cudnn
from icecream import ic
from colorama import Fore,Back,Style
warnings.filterwarnings('ignore')

train_iter = 0
val_iter = 0

trainSamples = len(urban_train) - (len(urban_train) % BATCH_SIZE)
val_samples = len(urban_val)
iterPerEpoch = len(urban_train_dataloader)
val_steps = len(urban_val_dataloader)

cudnn.benchmark
CUDA_LAUNCH_BLOCKING=1
model_checkpoint = "model" #name

model.train(True)
for epoch in range(NUM_EPOCHS):
    epoch_loss = 0
    IoU = 0

    for i, (inputs, targets) in enumerate(urban_train_dataloader):
        train_iter += 1
        optimizer_fn.zero_grad()

        # feeds in model
        inputs = inputs.to(DEVICE)
        labels = targets.to(device=DEVICE,dtype=torch.long)

        output_logits,_,_ = model(inputs)

        # compute loss
        loss = loss_fn(output_logits, labels)

        # backward loss and optimizer step
        loss.backward()
        optimizer_fn.step()

        #compute the training accuracy
        _, predicted = torch.max(output_logits.data, 1)

        intersection = torch.logical_and(labels, predicted)
        union = torch.logical_or(labels, predicted)
        print(intersection)
        print(union)
        IoU += torch.sum(intersection).item() / torch.sum(union).item()

        step_loss = loss.data.item()
        epoch_loss += step_loss
    avg_loss = epoch_loss/iterPerEpoch
    print(f"IoU:{IoU}, samples:{trainSamples}")
    avg_IoU = (IoU / trainSamples)
    print(Fore.GREEN + 'Val: Epoch = {} | Loss {:.3f} | mean-IoU = {:.3f}'.format(epoch + 1, avg_loss, avg_IoU))
    """
    #train_logger.add_epoch_data(epoch+1, trainAccuracy, avg_loss)

    if validate:
        if (epoch+1) % 1 == 0:
            model.train(False)
            val_loss_epoch = 0
            numCorr = 0
            for j, (inputs, targets) in enumerate(val_loader):
                val_iter += 1
                inputVariable = inputs.permute(1, 0, 2, 3, 4).to(DEVICE)
                labelVariable = targets.to(DEVICE)

                output_label, _ = model(inputVariable)
                val_loss = loss_fn(output_label, labelVariable)

                val_loss_step = val_loss.data.item()
                val_loss_epoch += val_loss_step
                _, predicted = torch.max(output_label.data, 1)
                numCorr += torch.sum(predicted == labelVariable.data).data.item()
                #val_logger.add_step_data(val_iter, numCorr, val_loss_step)

            val_accuracy = (numCorr / val_samples) * 100
            avg_val_loss = val_loss_epoch / val_steps

            print(Fore.GREEN + 'Val: Epoch = {} | Loss {:.3f} | Accuracy = {:.3f}'.format(epoch + 1, avg_val_loss, val_accuracy))
            if val_accuracy > min_accuracy:
                print("[||| NEW BEST on val||||]")
                save_path_model = os.path.join(model_folder, model_checkpoint)
                torch.save(model.state_dict(), save_path_model)
                min_accuracy = val_accuracy
"""
    optim_scheduler.step()


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
